In [1]:
%pip install BeautifulSoup4
%pip install markdownify

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import os
import requests
import time
from typing import Optional
from typing import Tuple
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup
from markdownify import markdownify as md
import pandas as pd

In [14]:
host = 'https://www.familysearch.org'
base_dir = 'data/raw'
bs_parser = 'html.parser'
delay_seconds = 5
site = 'source-linker-learning-center'

if not os.path.exists(base_dir):
    os.makedirs(base_dir)

In [15]:
# Function to get the article page from URL and return status code and response text as tuple of strings
def get_page(
    url: str,
    delay_seconds: int = 30,
    headers: Optional[dict[str, str]] = None,
    encoding: str = "utf-8",
    timeout: int = 30,
) -> Tuple[str, str]:
    if headers is None:
        headers = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Encoding": "gzip, deflate",
        "Accept-Language": "en-US,en;q=0.9",
        "Cache-Control": "no-cache",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
    }
    # Get Response from URL and return status code and response text
    response = requests.get(url, headers=headers, timeout=timeout)
    time.sleep(delay_seconds)
    if encoding:
        response.encoding = encoding
    return response.status_code, response.text


# Get Response from URL with raise exception if status code is not 200
def get_response_from_url(url: str, delay_seconds: int = 30) -> str:
    status_code, response = get_page(url, delay_seconds)
    if status_code != 200:
        raise Exception(f"Failed to get response from {url}")
    return response

In [16]:
def _is_article(url):
    """A talk URL has 6 components (first component is empty) and last component does not end in -session.
    """  
    path_components = urlparse(url).path.split('/')
    return len(path_components) == 6 and not path_components[-2].endswith('fieldops')

In [17]:
def get_article_urls(base_url):
    """Find and return article URLs from the base URL."""
    dir_html = get_response_from_url(base_url, delay_seconds)
    soup = BeautifulSoup(dir_html, bs_parser)
    return set(urljoin(base_url, a['href']) for a in soup.find_all('a',href=True)\
        if _is_article(urljoin(base_url, a['href'])))

In [18]:
# Get URLs from the host page and save them to an array
dir_url = f"{host}/en/help/helpcenter/{site}"
urls = list(get_article_urls(dir_url)) 
urls.append(dir_url) 
# for u in urls:
#     print(f"{u}\n")
print(len(urls), urls)

23 ['https://www.familysearch.org/en/help/helpcenter/article/understanding-the-focus-person-in-source-linker', 'https://www.familysearch.org/en/help/helpcenter/article/entering-a-reason-statement', 'https://www.familysearch.org/en/help/helpcenter/article/using-source-linker-to-attach-sources', 'https://www.familysearch.org/en/help/helpcenter/article/how-tagging-works', 'https://www.familysearch.org/en/help/helpcenter/article/what-is-a-source-box', 'https://www.familysearch.org/en/help/helpcenter/article/editing-information-in-your-family-tree', 'https://www.familysearch.org/en/help/helpcenter/article/what-to-do-when-the-names-in-source-linker-dont-match', 'https://www.familysearch.org/en/help/helpcenter/article/view-records-and-tree-profiles-without-leaving-source-linker', 'https://www.familysearch.org/en/help/helpcenter/article/what-are-record-hints-in-family-tree', 'https://www.familysearch.org/en/help/helpcenter/article/understanding-the-source-linker-screen-layout', 'https://www.fa

In [22]:
# Leer el contenido de faq.txt con codificación utf-8
try:
    with open('faq.txt', 'r', encoding='utf-8') as file:
        content = file.read()
except UnicodeDecodeError:
    # Si utf-8 no funciona, intenta con una codificación alternativa
    with open('faq.txt', 'r', encoding='latin1') as file:
        content = file.read()

# Dividir el contenido en secciones
sections = content.split('Section ')[1:]  # Dividir por secciones

faq_dict = {}
for section in sections:
    lines = section.split('\n')
    section_title = lines[0].strip()  # Título de la sección
    q_a_pairs = []
    question = None
    
    for line in lines[1:]:
        line = line.strip()
        if line.startswith('What') or line.startswith('How') or line.startswith('Why'):
            if question:
                q_a_pairs.append((question, answer))
            question = line
            answer = ''
        elif line.startswith('•') or line:
            answer += (line + ' ')
    
    if question:
        q_a_pairs.append((question, answer.strip()))
    
    faq_dict[section_title] = q_a_pairs

# Verificar la estructura
for section, qas in faq_dict.items():
    print(f"Section: {section}")
    for question, answer in qas:
        print(f"Q: {question}\nA: {answer}\n")

Section: 1: Tagging Sources in Source Linker
Q: What does tagging a source do in Source Linker?
A: Tagging a source turns it into a reference for details listed on the person page in Family Tree. Where can you find the option to create a tag? It appears in the left column of Source Linker after clicking the purple Compare button. 

Q: What categories can you tag a source to?
A: Name, Birth, Death, Sex, Christening, Burial, Residence. 

Q: How do you tag a source to a detail?
A: Click the checkbox next to the detail. 

Q: How can you remove a source from a detail?
A: Uncheck the checkbox next to the detail.

Section: 2: Understanding the Source Linker Screen Layout
Q: What does the left column in Source Linker display?
A: Details from the historical record. 

Q: What does the right column in Source Linker display?
A: Details from the ancestor’s person page in Family Tree. 

Q: What should you do if the ancestor is not in Family Tree?
A: You will see an option to add the ancestor.

Secti

In [27]:
# Crear una lista para el DataFrame
data = []
for url in urls:
    for section_title, qas in faq_dict.items():
        for question, answer in qas:
            data.append([url, question, answer])

# Crear DataFrame y guardar en CSV
df = pd.DataFrame(data, columns=['url', 'question', 'answer'])
df.to_csv('faq_with_urls.csv', index=False)
df.head()

,url,question,answer
0,https://www.familysearch.org/en/help/helpcente...,What does tagging a source do in Source Linker?,Tagging a source turns it into a reference for...
1,https://www.familysearch.org/en/help/helpcente...,What categories can you tag a source to?,"Name, Birth, Death, Sex, Christening, Burial, ..."
2,https://www.familysearch.org/en/help/helpcente...,How do you tag a source to a detail?,Click the checkbox next to the detail.
3,https://www.familysearch.org/en/help/helpcente...,How can you remove a source from a detail?,Uncheck the checkbox next to the detail.
4,https://www.familysearch.org/en/help/helpcente...,What does the left column in Source Linker dis...,Details from the historical record.
